<a href="https://colab.research.google.com/github/ivmarchuk/get-yt-trends/blob/main/youtube-ET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time, datetime
from datetime import datetime
import os
import configparser

In [ ]:
# country_codes = ['US','GB','RU','PL','DE']
today = datetime.now()
today.strftime('%Y-%m-%d')
today
print(today)

2022-07-25 10:35:19.833652


In [ ]:
video_list = ['rolT_uwCJuo','EZK6ELyGxr4','Kuqcl5rso7U','nQDgVjd6yyI','47ynjeFgZMs','TlH5nhv6v6w','m4c4tNg6wzo','4KW7FekWWVU']

In [ ]:
def get_dislikes(df, df_dis, video_id: str):
  try:
    url = 'https://returnyoutubedislikeapi.com/votes?videoId='+video_id
  except Exception as e:
    print('Something went wrong with loading dislikes data: ', e)
        
  dis_req = requests.get(url).json()
  for dis in dis_req:
    dislike_count = dis_req['dislikes']
    rating = dis_req['rating'] 
    append_features = {'video_id': video_id,
                              'dislike_count': dislike_count,
                              'rating': rating}
    df_dis = df_dis.append(append_features, ignore_index = True)
      

  return df_dis


for video_id in video_list: 
  df_dis.append(get_dislikes(df=df,df_dis=df_dis,video_id=video_id))

In [ ]:
def get_categories(df_cat, API_KEY):
    try: 
        url = 'https://www.googleapis.com/youtube/v3/videoCategories?part=snippet&regionCode=PL&key='+API_KEY
    except Exception as e:
        print('Something went wrong with loading categories data: ', e)
    finally:
        print('Loading categories finished: ')

    cat_req = requests.get(url).json()
    for item in cat_req['items']: 
        category_id = item['id']
        category_name = item['snippet']['title']

        append_features = {"category_id" : category_id,
                        'category_name' : category_name}

        df_cat = df_cat.append(append_features,  ignore_index = True)
    
    return df_cat

In [ ]:
def merge_datasets(df, df_dis, df_cat):
  df_n = pd.merge(df, df_dis, on = 'video_id')
  df_merged = pd.merge(df_n, df_cat, on = 'category_id')

  # add columns for future data validation check
  df_merged['likes_availability'] = True
  print('Merging datasets finished', today)

  # TBA instead of nulls
  df_merged = df_merged.fillna('TBA')
  return df_merged


In [ ]:
# df to csv
def df_to_csv(df): 
  file_csv = '/SAMPLE/dataset.csv'
  df.to_csv(file_csv, sep=';', encoding='utf-8')

  return file_csv